In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["KERAS_BACKEND"] = "torch"

import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

import keras
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Rescaling, Dropout, Resizing
from keras.layers import RandomFlip, RandomTranslation, RandomRotation, RandomZoom, MultiHeadAttention, AugMix, RandAugment
from keras.models import Sequential
from keras.metrics import F1Score, Precision, AUC
from keras.losses import CategoricalFocalCrossentropy, BinaryFocalCrossentropy
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.optimizers.schedules import ExponentialDecay, CosineDecay
from keras.optimizers import AdamW

from sklearn.metrics import average_precision_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE, ADASYN, KMeansSMOTE
from imblearn.under_sampling import TomekLinks, RepeatedEditedNearestNeighbours
from imblearn.combine import SMOTETomek, SMOTEENN
from sklearn.utils import compute_class_weight

from PIL import Image


In [11]:
@keras.saving.register_keras_serializable()
def mean_average_precision(y_true, y_pred):
    aps = []
    for c in range(7):
        ap = average_precision_score(y_true.detach().cpu().numpy()[:, c], y_pred.detach().cpu().numpy()[:, c])
        aps.append(ap)

    mAP = sum(aps) / len(aps)
    return mAP

In [55]:
model = torch.load('model_v2b0_224px_f1_norandaug.pth', weights_only=False)

In [58]:
with Image.open('/home/dimitri/Downloads/nod.jpg') as im:
    arr = np.asarray(im.resize((224,224)))

In [59]:
model.predict(arr[np.newaxis, :, :, :])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step


array([[4.5810252e-06, 9.5838932e-06, 1.4906007e-04, 1.0585163e-08,
        9.9420220e-01, 5.5612214e-03, 7.3227231e-05]], dtype=float32)

In [38]:
class_mapping = {
    "akiec": "canc",
    "bcc": "canc",
    "bkl": "nocanc",
    "df": "nocanc",
    "mel": "canc",
    "nv": "nocanc",
    "vasc": "nocanc"
}

dict_label = {k:v for k,v in zip(list(class_mapping.keys()), range(7))}

In [39]:
dict_label

{'akiec': 0, 'bcc': 1, 'bkl': 2, 'df': 3, 'mel': 4, 'nv': 5, 'vasc': 6}

In [51]:
df = pd.read_csv('/home/dimitri/projects/milk10k/MILK10k_Training_GroundTruth.csv')

In [52]:
df.sum(axis=0)

lesion_id    IL_0000652IL_0003176IL_0004688IL_0005081IL_000...
AKIEC                                                    303.0
BCC                                                     2522.0
BEN_OTH                                                   44.0
BKL                                                      544.0
DF                                                        52.0
INF                                                       50.0
MAL_OTH                                                    9.0
MEL                                                      450.0
NV                                                       746.0
SCCKA                                                    473.0
VASC                                                      47.0
dtype: object

In [54]:
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello_world():
    return "<p>Hello, World!</p>"

In [4]:
label_to_name = {
    0: 'Actinic keratoses',
    1: 'Basal cell carcinoma',
    2: 'Benign keratosis-like lesions',
    3: 'Dermatofibroma',
    4: 'Melanoma',
    5: 'Melanocytic nevi',
    6: 'Vascular lesions'
    }  

In [32]:
model = torch.load('/home/dimitri/projects/SK-MNIST/model_v2b0_224px_f1_norandaug.pth', weights_only=False)
with Image.open("/home/dimitri/Downloads/common_nevus.png.webp") as im:
    arr = np.asarray(im.resize((224,224)))
pred = model.predict(arr[np.newaxis,:,:,:])
result_dict = {k:v.item()*100 for k,v in zip(list(label_to_name.values()), list(pred.ravel()))}
result_dict = dict(sorted(result_dict.items(), key=lambda x:x[1], reverse=True))
result = {k:v for k,v in list(result_dict.items())[:3]}
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
{'Melanocytic nevi': 95.84264159202576, 'Melanoma': 1.2666101567447186, 'Basal cell carcinoma': 1.2469535693526268}


In [33]:
list(result_dict.items())

[('Melanocytic nevi', 95.84264159202576),
 ('Melanoma', 1.2666101567447186),
 ('Basal cell carcinoma', 1.2469535693526268),
 ('Benign keratosis-like lesions', 1.0607588104903698),
 ('Vascular lesions', 0.563065055757761),
 ('Actinic keratoses', 0.010831988765858114),
 ('Dermatofibroma', 0.009144975047092885)]

In [18]:
pred.shape

(1, 7)